In [1]:
#Vinicius de Barros Silva 10335913

In [2]:
url='http://www.lps.usp.br/hae/apostila/feiCorCrop.zip'
import os; nomeArq=os.path.split(url)[1]
if not os.path.exists(nomeArq):
 print("Baixando o arquivo",nomeArq,"para diretorio default",os.getcwd())
 os.system("wget -nc -U 'Firefox/50.0' "+url)
else:
 print("O arquivo",nomeArq,"ja existe no diretorio default",os.getcwd())
print("Descompactando arquivos novos de",nomeArq) 
os.system("unzip -u "+nomeArq)

Baixando o arquivo feiCorCrop.zip para diretorio default /content
Descompactando arquivos novos de feiCorCrop.zip


0

In [3]:
#resnet_transf.py 
#Faz transfer learning usando ResNet50.
#https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-keras-b941601fcad5
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np; 
import tensorflow.keras as keras; import keras.backend as K;
from tensorflow.keras import optimizers, callbacks, regularizers;
from tensorflow.keras.regularizers import l2;
from tensorflow.keras.models import Sequential, Model;
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten;
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

def leCsv(nomeDir, nomeArq, nl=0, nc=0):
  st=os.path.join(nomeDir,nomeArq); 
  arq=open(st,"rt"); lines=arq.readlines(); arq.close(); n=len(lines)

  linhas_separadas=[]
  for linha in lines:
    linha=linha.strip('\n'); linha=linha.split(';'); linhas_separadas.append(linha);

  ay=np.empty((n),dtype='float32'); ax=np.empty((n,nl,nc,3),dtype='float32');
  for i in range(len(linhas_separadas)):
    linha=linhas_separadas[i];
    img_path=os.path.join(nomeDir,linha[0])
    t = image.load_img(img_path, target_size=(nl,nc))
    x = image.img_to_array(t)
    x = np.expand_dims(x, axis=0)
    ax[i] = preprocess_input(x)
    ay[i] = np.float32(linha[1]); #0=m ou 1=f
  return ax, ay;

#<<<<<<<<<<<<<<<<<<<<<<<< main <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
nomeprog="resnet_transf"
#Original: 280x200, redimensionado: 224x224
num_classes=2; nl=224; nc=224
diretorioBd="."
ax, ay = leCsv(diretorioBd,"treino.csv", nl=nl, nc=nc); #200 imagens
qx, qy = leCsv(diretorioBd,"teste.csv", nl=nl, nc=nc);  #100 imagens
vx, vy = leCsv(diretorioBd,"valida.csv", nl=nl, nc=nc);  #100 imagens
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)
vy = keras.utils.to_categorical(vy, num_classes)

input_shape = (nl,nc,3); batch_size = 10; 
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model.summary()
x = base_model.output
camada = keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1,1), padding="same")
x = camada(x)
x = Flatten()(x)
predictions = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

#Nao permite treinar base_model. So as camadas densas sao treinadas:
for layer in base_model.layers: layer.trainable = False
#Treina com learning rate grande
otimizador=keras.optimizers.Adam(learning_rate=1e-3)
model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))

score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)

#Libera todos layers do model para treinar:
for layer in model.layers: layer.trainable = True
#Treina com learning rate pequena todas as camadas
otimizador=keras.optimizers.Adam(learning_rate=1e-6)
model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))

score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)


94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/5
20/20 - 12s - loss: 0.7577 - accuracy: 0.8850 - val_loss: 0.4600 - val_accuracy: 0.9000 - 12s/epoch - 607ms/step
Epoch 2/5
20/20 - 1s - loss: 0.0458 - accuracy: 0.9900 - val_loss: 0.6042 - val_accuracy: 0.9300 - 1s/epoch - 50ms/step
Epoch 3/5
20/20 - 1s - loss: 0.0113 - accuracy: 0.9900 - val_loss: 0.1382 - val_accuracy: 0.9700 - 993ms/epoch - 50ms/step
Epoch 4/5
20/20 - 1s - loss: 2.8864e-06 - accuracy: 1.0000 - val_loss: 0.2071 - val_accuracy: 0.9800 - 1s/epoch - 50ms/step
Epoch 5/5
20/20 - 1s - loss: 9.2828e-04 - accuracy: 1.0000 - val_loss: 0.1562 - val_accuracy: 0.9800 - 1s/epoch - 50ms/step
Training loss: [6.085462018745602e-07, 1.0]
Validation loss: [0.15620669722557068, 0.9800000190734863]
Test loss: [1.2678608894348145, 0.9100000262260437]
Epoch 1/5
20/20 - 10s - loss: 9.1710 - accuracy: 0.5750 - val_loss: 0.1662 - val_accuracy: 0.9800 - 10s/epoch - 513ms/step
Epoch 2/5
20/20 - 3s - loss: 6.3097 - accur

In [4]:
model.save('/content/drive/My Drive/Hae/aula10/resnet_transf.h5')